# Barycentric interpolation

## Recap: Lagrange interpolation

Lagrange interpolation is a way to compute a polynomial based on a set of points. 
In the context of ZKP, sometimes we need to interpolate a multiplicative subgroup to the trace. 
Let us do a quick recap. Here is our formula.

$$
L(x):=\sum_{j=0}^ky_j\ell_j(x)
$$

We call $\ell_j(x)$ the **Lagrange base**. In its natural form, it looks like this:

$$
\ell_j(x):=\prod_{i=0,}^k\frac{x-x_i}{x_j-x_i}=\frac{(x-x_0)}{(x_j-x_0)}\cdots\frac{(x-x_{j-1})}{(x_j-x_{j-1})}\frac{(x-x_{j+1})}{(x_j-x_{j+1})}\cdots\frac{(x-x_k)}{(x_j-x_k)}
$$

The first thing to note is that for each Lagrange base, $j \neq i$ .

The second thing is that the base acts like a **selector/filter**,
which equals 1 when  x = x_j , and equals 0 otherwise.

The function looks large, but don't be afraid.
so let’s do a small exercise to help you understand it.

Suppose we have three points (2, 5) (4, 8) (8, 11) which are field elements in $F_{13}$,
and we want to find the polynomial on this field.

First, we calculate the Lagrange base.

We have 

$$
\ell_0(x):=\prod_{i=1,}^2\frac{x-x_i}{x_0-x_i}=\frac{(x-x_1)}{(x_0-x_1)}\frac{(x-x_2)}{(x_0-x_2)}
$$

and we know $x_1 = 4$ and $x_2 = 8$ then we have (you need to keep in mind we are doing the calculation under finite filed)

$$
\ell_0(x):=\prod_{i=1,}^2\frac{x-x_i}{x_0-x_i}=\frac{(x-4)}{(2-4)}\frac{(x-8)}{(2-8)}=\frac{x^2 + x + 6}{12}= 12x^2 + 12x + 7
$$

Try to calculate $\ell_1(x)$ and $\ell_2(x)$ by yourself.
I won’t show you the process, but here are the results for you to check:

$$
\ell_1(x) = 8x^2 + 11x + 11
$$

$$
\ell_2(x) = 6x^2 + 3x + 9
$$

Now, we calculate:

$$
L(x):=\sum_{j=0}^ky_j\ell_j(x) = 5 * \ell_0(x) + 8 * \ell_1(x) + 11 * \ell_2(x) = 8x^2 + 12x + 1
$$

Now we have finished the recap of Lagrange interpolation.
Let’s consider the following question:

What should we do if we have a new point (3, 5) to interpolate?

In a native way, we need to re-calculate all the lagrange base again.
How to avoid this?

## Barycentric interpolation

Don’t panic, the Barycentric interpolation is just a simple transformation from lagrange interpolation

First, let's rewrite the lagrange base. We use $\ell(x)=(x-x_0)(x-x_1)\cdots(x-x_k)$ (which include all indices)

We can get 

$$
\ell_j(x)=\frac{\ell(x)}{x-x_j}\frac1{\prod_{i=0,i\neq j}^k(x_j-x_i)}
$$

then we define barycentric weights

$$
w_j=\frac1{\prod_{i=0,i\neq j}^k(x_j-x_i)}
$$

with above notion we can rewrite our Lagrange interpolation in to following form

$$
L(x)=\ell(x)\sum_{j=0}^k\frac{w_j}{x-x_j}y_j
$$

We alse notice one thing, if you plus all barycentric weights together, you will find the result will be 1

$$
\forall x,\mathrm{~}g(x)=\ell(x)\sum_{j=0}^k\frac{w_j}{x-x_j}
$$

Use above function as dominator of $L(x)$ we will get second (true) formof the barycentric formula

$$
L(x)=\frac{\sum_{j=0}^k\frac{w_j}{x-x_j}y_j}{\sum_{j=0}^k\frac{w_j}{x-x_j}}
$$

Let’s look at barycentric weights. These values depend only on the evaluation points, not the specific polynomial.
By transforming the Lagrange interpolation into barycentric interpolation,
we can precompute the barycentric weights once and store them to separate the heavy computation part from the whole interpolation process.

We can find it in the following Plonky3 code

```rust
// interpolation/src/lib.rs
    let col_scale: Vec<_> = if let Some(diff_invs) = diff_invs {
        g.zip(diff_invs)
            .map(|(sg, &diff_inv)| diff_inv * sg)
            .collect()
    } else {
        let subgroup = g.collect::<Vec<_>>();
        let diffs: Vec<EF> = subgroup
            .par_iter()
            .map(|&subgroup_i| point - subgroup_i * shift)
            .collect();
        let diff_invs = batch_multiplicative_inverse(&diffs);
        subgroup
            .par_iter()
            .zip(diff_invs)\\
            .map(|(&sg, diff_inv)| diff_inv * sg)
            .collect()
    };
```

## Under the two adic field

In many applications of barycentric evaluation, we can choose the evaluation points. One very convenient choice is the set $\{1,\omega,\omega^2\ldots\omega^{N-1}\}$ $N$ is a power of two. The main advantage of this choice is that it removes the need to pre-compute the $w_j$ values, because there is a simple closed-form value for them.

For a 2-adic subgroup we have following feature:
$$
z_H(X)=\prod_{i=0}^{N-1}(X-\omega^i)=X^N-1
$$

"Proof": 

For $F_{13}$ we have $F_{13}=(0,1,2,3,4,5,6,7,8,9,10,11,12)$

The the generator of the multiplicative group is $g=2$, as $13-1 = 2 * 2 * 3$, we have a smaller group of order 4, which generator is 5

$H=(\omega^0=1,\omega^1=5,\omega^2=12,\omega^3=8)$

Suppose N = 4

$$
\begin{aligned}&(X-\omega^0)(X-\omega^1)(X-\omega^2)(X-\omega^3)\\&=\quad(X-1)(X-\omega)(X+1)(X-\omega^3)\\&=\quad(X^2-1)(X-\omega)(X+\omega)\\&=\quad(X^2-1)(X^2-\omega^2)\\&=\quad(X^2-1)(X^2+1)\\&=\quad(X^4-1)\end{aligned}
$$

You should calculate above process by yourself (Do not forget to compute mod 13)

And we can find $w_j=\frac1{\prod_{i=0,i\neq j}^k(x_j-x_i)}$ has a similar structure. we can apply above optimization on $w_j$, finally we will get 

$$
w_i=\frac N{\omega^i}
$$

You can read this blog for more information https://hackmd.io/@vbuterin/barycentric_evaluation


## REF

https://hackmd.io/@vbuterin/barycentric_evaluation

https://github.com/sec-bit/learning-zkp/blob/master/plonk-intro-zh/2-plonk-lagrange-basis.md

https://epubs.siam.org/doi/epdf/10.1137/S0036144502417715